# Examples of how to create and use databases with Python

These examples focus on how to use python to make working with databases easier, and evaluate what options there are for data storage. One essential aspect of python create a persistent session, where datastorage becomes a necessity. Storing data in Python can be done in a variety of ways using pickle, sqlite3, sqlalchemy, csv, xml, xlsx, txt, mongodb, etc. One big advantage of having a database rather than a file is to allow for concurrent access, or allow for complex searches

### Terminology
* flat files - csv, txt, etc
* DBM keyed files - keyed access to dictionary like files
* pickled objects-serialized python objects saved to files and streams. python module to store objects
* shelve files - pickled python objects saved in DBM files
* OODB - Object oriented databases, ZODB, Durus
* RDBMS - SQL relational databases (eg SQLite). MySQL and Postgres) is open source
* ORM - Object relational mappers, SQLObject, SqlAlchemy retains python class view but also translates the information to and from a relational database

The module sqlite3 provides access to the SQLite relational database and provides 
a large portion of the functionality of SQL

### References
Python for Data Anaylsis- Wes McKinney <br />
Beginning Python_ From Novice to Profess - Magnus Lie Hetland <br />
Data Analysis with Open Source  - Philipp K. Janert <br />
Data Structures and Algorithms  - Michael T. Goodrich <br />
Learning Python - Mark Lutz <br />

## Python for Data Anaylsis- Wes McKinney
http://blog.wesmckinney.com/
<img src='http://akamaicovers.oreilly.com/images/0636920023784/cat.gif'>


In [137]:
import sqlite3
import pandas

In [138]:
# create database in memory, pg 175
query = """
CREATE TABLE test
(a VARCHAR(20),b VARCHAR(20),
c REAL, d INTEGER
);"""
con = sqlite3.connect(':memory:')  # creates a temporary data in memory
con.execute(query)
con.commit()

In [139]:
# insert some data
data = [('Atlanta', 'Georgia', 1.25,6),
        ('Tallahassee','Florida',2.6,2),
        ('Sacramento','California', 1.7, 5)]
stmt = "INSERT INTO test VALUES(?,?,?,?)"
con.executemany(stmt, data)
con.commit()

In [140]:
# return data
cursor = con.execute('select * from test')
rows = cursor.fetchall()
rows

[(u'Atlanta', u'Georgia', 1.25, 6),
 (u'Tallahassee', u'Florida', 2.6, 2),
 (u'Sacramento', u'California', 1.7, 5)]

In [141]:
cursor.description

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [128]:
# create a pandas dataframe from the database
pandas.DataFrame(rows, columns=zip(*cursor.description)[0])

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,2
2,Sacramento,California,1.70,5


In [129]:
# directly read the database
pandas.io.sql.read_sql('select * from test',con)

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,2
2,Sacramento,California,1.70,5


Can also use a NoSQL database such as BerkeleyDB or MongoDB

## Programming Python - Mark Lutz.pdf
http://www.rmi.net/~lutz/
<img src= 'http://www.rmi.net/~lutz/ora-pp4e-small.gif'>

pg 1329 <br/>
SQL databases can be accessed directly from python with some basic functionality

* connection objects represent a connection to a database
* cursor objects - represents an SQL statement submitted as a string
* Query results - uses select statement

In [164]:
import sqlite3
conn = sqlite3.connect('dbase1')

In [165]:
# making a database and tables
curs = conn.cursor()
tblcmd = '''CREATE TABLE people 
            (name, CHAR(30), job CHAR(10), pay INT(4) )'''
curs.execute(tblcmd)

OperationalError: near "(": syntax error

In [166]:
conn.close()

## Data Analysis with Open Source  - Philipp K. Janert
<img src='http://akamaicovers.oreilly.com/images/9780596802363/cat.gif?'>

In [130]:
import sqlite3, os

In [131]:
# delete the database if it exists
try:
    conn.close() # disconnect database
    os.remove('data.dbl')
except:
    pass

In [132]:
# connect and obtain a cursor
conn = sqlite3.connect('data.dbl')
conn.isolation_level = None   # use autocommit
c = conn.cursor()

In [133]:
# Create tables
c.execute( """CREATE TABLE orders
                ( id INTEGER PRIMARY KEY AUTOINCREMENT,
                customer )""" )
c.execute( """CREATE TABLE lineitems
            ( id INTEGER PRIMARY KEY AUTOINCREMENT,
            orderid, description, quantity )""" )

In [134]:
# Insert Values
c.execute("INSERT INTO orders (customer) VALUES ('Joe Blo')")
id = str(c.lastrowid)

c.execute("""INSERT INTO lineitems (orderid, description, quantity )
             VALUES ( ?, 'Widget 1','2')""",(id,))
c.execute("""INSERT INTO lineitems (orderid, description, quantity )
             VALUES ( ?, 'Fidget 1','1')""",(id,))
c.execute("""INSERT INTO lineitems (orderid, description, quantity )
             VALUES ( ?, 'Part 17','5')""",(id,))

c.execute("INSERT INTO orders ( customer ) VALUES ('Jane Doe')" )
id = str(c.lastrowid)

c.execute("""INSERT INTO lineitems (orderid, description, quantity )
             VALUES ( ?, 'Fidget 2','3')""",(id,))
c.execute("""INSERT INTO lineitems (orderid, description, quantity )
             VALUES ( ?, 'Part 9','2')""",(id,))


In [135]:
# Query
c.execute("""SELECT li.description FROM orders o, lineitems li
            WHERE o.id = li.orderid AND o.customer LIKE '%Blo' """)
for r in c.fetchall():
    print[0]
    
c.execute( """SELECT orderid, sum(quantity) FROM lineitems
                GROUP BY orderid ORDER BY orderid desc""" )
for r in c.fetchall():
    print "OrderID: ", r[0], "\t Items: ", r[1]

[0]
[0]
[0]
OrderID:  2 	 Items:  5
OrderID:  1 	 Items:  8


In [136]:
# disconnect database
conn.close()
# os.remove('data.dbl')